In [14]:
# !ls data/

In [15]:
import pandas as pd
from collections import Counter
import os
import glob
import re
import shutil
from matplotlib import pyplot as plt
import numpy as np

In [16]:
def count_null(df):
    return df.isnull().sum()
def columns_fix(df):
    try:
        df_ouro.drop('REF_ANOMES', inplace=True, axis=1)
        df_ouro['pagina_dodf'] = df_ouro.pagina_dodf.str.replace(
            r'([^\d]+)',''
        )
    except:
        pass
    df_ouro.rename(str.lower, inplace=True, axis=1)
    df_ouro.rename(lambda x: x.replace('_dodf', ''), axis=1, inplace=True)
    df_ouro.rename(lambda x: x.replace('matricula', 'mat'), axis=1, inplace=True)

def postprocess(df):
    df['pagina'] = df.pagina.str.replace(r'([^\d]+)', '')
    to_drona_drop = [
        'cod_mat_sigrh',
        'nome_sigrh',
        'fund_legal',
        'cargo',
        'ato',
        
    ]
    df.dropna(subset=to_drona_drop, inplace=True)
    df['data'] = pd.to_datetime(df.data)

In [17]:
df_ouro = pd.read_csv(
    'data/aposentadoria-ouro/aposentaria-ouro-®.csv', sep='®',
    engine='python',
    index_col=False,
)
# count_null(df_ouro)
columns_fix(df_ouro)
postprocess(df_ouro)
# df_ouro.columns, df_ouro.dtypes

In [18]:
matters = ['num','data', 'nome_ato',
        'ato', 'pagina',
        'cod_mat_ato',
        'cod_mat_sigrh', 'tipo', 'processo'
        ]
df_zip = df_ouro[matters]
df_zip[df_zip.cod_mat_sigrh == '01007238']
# df_zip.head()

,num,data,nome_ato,ato,pagina,cod_mat_ato,cod_mat_sigrh,tipo,processo
116,68,2018-04-10,CLAUDIA CRISTIANE CAVALCANTE AIELLO,CONCEDER APOSENTADORIA,26,1660377X,01007238,NORMAL,°00070.00011832/2018-38


In [19]:
df_zip.ato.value_counts(),df_zip.tipo.value_counts()

(CONCEDER APOSENTADORIA    4719
 APOSENTAR                  766
 Name: ato, dtype: int64,
 NORMAL          5442
 EDICAO EXTRA      42
 SUPLEMENTO         1
 Name: tipo, dtype: int64)

In [20]:
gp = df_zip.groupby(['data', 'num'])
# ks=[i for i in dir(gp) if not i.startswith('_')]
date_num = gp.count().iloc[:, [0]].sort_values(
    by=['nome_ato', 'data', 'num'], ascending=False)

# Arquivo com datas e mnumeros dos DODFs pra baixar
# cada qual contendo 50+ ocorrencias
date_num = date_num[date_num.iloc[:, 0] > 50]
date_num.reset_index(inplace=True)
date_num = date_num.rename({'nome_ato':'qtd_miner'}, axis=1)
date_num['data'] = date_num.data.map(lambda x: x.strftime('%d-%m-%Y'))
date_num.sort_values('data').to_csv(
    'data/date_num_gt50.csv',index=False
);
date_num.head(2)

,data,num,qtd_miner
0,05-09-2019,169,131
1,03-07-2018,124,116


In [21]:
# Arquivo com datas e mnumeros dos DODFs pra baixar
# cada qual contendo 50+ ocorrencias
gp = df_zip.groupby(['data', 'num'])
# ks=[i for i in dir(gp) if not i.startswith('_')]
date_num = gp.count().iloc[:, [0]].sort_values(
    by=['data', 'num', 'nome_ato', ], ascending=False)

date_num.reset_index(inplace=True)
date_num = date_num.rename({'nome_ato':'qtd_miner'}, axis=1)
date_num['data'] = date_num.data.map(lambda x: x.strftime('%d-%m-%Y'))
date_num.sort_values('data').to_csv(
    'data/date_num.csv',index=False
);
date_num.head(2)

,data,num,qtd_miner
0,30-09-2019,186,66
1,26-09-2019,184,8


In [22]:

# # Arquivo com datas e mnumeros dos DODFs pra baixar
# # cada qual contendo 50+ ocorrencias
# date_num = date_num[date_num.iloc[:, -1] > 50]
# # date_num.reset_index(inplace=True)
# date_num = date_num.rename({'nome_ato':'qtd'}, axis=1)
# date_num['data'] = date_num.data.map(lambda x: x.strftime('%d-%m-%Y'))
# date_num.sort_values('data').to_csv('data/date_num.csv', index=False);
# date_num

# # print("TOTAL (>50):", date_num[date_num.nome_ato > 50].sum()[0],end=' ')


In [23]:
# df_a = []
# df_sea = []
# for i in glob.glob('data/extraido/*.csv'):
#     f = i.split('/')[-1]
#     df = pd.read_csv(i, index_col=False)
#     df.drop('Unnamed: 0', axis=1, inplace=True)
#     df['num_date'] = df.shape[0] * [re.search(r'(\d.+\d)(?=[.])',f).group()]
#     if f.startswith('aposentadoria'):
#         df_a.append(df)
#     else:
#         df_sea.append(df)
# df_a = pd.concat(df_a)
# df_sea = pd.concat(df_sea)